In [1]:
import numpy as np
import xarray as xr

In [2]:
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid.nc')
up_eta_rho = grid_ds.dims['eta_rho']
up_xi_rho = grid_ds.dims['xi_rho']

## Fix mask_rho variable - boundaries check

In [4]:
border = {
    'west': grid_ds.mask_rho.isel(xi_rho=slice(0, 2)),
    'north': grid_ds.mask_rho.isel(eta_rho=slice(up_eta_rho-2, up_eta_rho)),  # grid_ds.mask_rho[-2:, :]
    'east': grid_ds.mask_rho.isel(xi_rho=slice(up_xi_rho-2, up_xi_rho)),
    'south': grid_ds.mask_rho.isel(eta_rho=slice(0, 2)),
}

pattern = {
    'west': np.array([[1., 0.]]),
    'north': np.array([[0.], [1.]]),
    'east': np.array([[0., 1.]]),
    'south': np.array([[1.], [0.]]),
}

def find_matches(border_position: str):
    """
    find indices of matches
    """
    if border_position in ('north', 'south'):
        idx = np.argwhere(np.asarray(border[border_position].values == pattern[border_position], 
                          dtype=np.int32).transpose() @ np.array([1, 1]) > 1)
    elif border_position in ('west', 'east'):
        idx = np.argwhere(np.asarray(border[border_position].values == pattern[border_position], 
                          dtype=np.int32) @ np.array([1, 1]) > 1)
    else:
        raise ValueError

    print(f"{border_position} indices: {idx.transpose()[0]}")
    return idx

In [5]:
west_idx = find_matches('west')
north_idx = find_matches('north')
east_idx = find_matches('east')
south_idx = find_matches('south')

west indices: []
north indices: [1345 1346 1349]
east indices: [695 752 753 906 940 941 951]
south indices: [212 226 227 228 371 384 488 489 490 491 492 564 565 566]


In [6]:
def get_values(eta_rho, xi_rho):
    try:
        return grid_ds.mask_rho.isel(eta_rho=eta_rho, xi_rho=xi_rho).values
    except IndexError:
        return np.NAN


def print_values():
    print(f"West values: {get_values(eta_rho=west_idx.squeeze(), xi_rho=0)}")
    print(f"North values: {get_values(eta_rho=up_eta_rho-1, xi_rho=north_idx.squeeze())}")
    print(f"East values: {get_values(eta_rho=east_idx.squeeze(), xi_rho=up_xi_rho-1)}")
    print(f"South values: {get_values(eta_rho=0, xi_rho=south_idx.squeeze())}")


In [7]:
print_values()

West values: nan
North values: [1. 1. 1.]
East values: [1. 1. 1. 1. 1. 1. 1.]
South values: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Get xi and eta coordinates to use loc assignment (isel or iloc won't work)

In [8]:

xi_rho_west_coords = grid_ds.coords['xi_rho'][0]
eta_rho_west_coords = grid_ds.coords['eta_rho'][west_idx.squeeze()] * xr.ones_like(xi_rho_west_coords)
print(f"Xi west coordinates: {xi_rho_west_coords}\n",
      f"Eta west coordinates: {eta_rho_west_coords}\n")

xi_rho_north_coords = grid_ds.coords['xi_rho'][north_idx.squeeze()]
eta_rho_north_coords = grid_ds.coords['eta_rho'][-1] * xr.ones_like(xi_rho_north_coords)
print(f"Xi north coordinates: {xi_rho_north_coords}\n",
      f"Eta north coordinates: {eta_rho_north_coords}\n")

xi_rho_east_coords = grid_ds.coords['xi_rho'][-1]
eta_rho_east_coords = grid_ds.coords['eta_rho'][east_idx.squeeze()] * xr.ones_like(xi_rho_east_coords)
print(f"Xi east coordinates: {xi_rho_east_coords}\n",
      f"Eta east coordinates: {eta_rho_east_coords}\n")

xi_rho_south_coords = grid_ds.coords['xi_rho'][south_idx.squeeze()]
eta_rho_south_coords = grid_ds.coords['eta_rho'][0] * xr.ones_like(xi_rho_south_coords)
print(f"Xi south coordinates: {xi_rho_south_coords}\n",
      f"Eta south coordinates: {eta_rho_south_coords}\n")


Xi west coordinates: 0.0
 Eta west coordinates: []

Xi north coordinates: [215200. 215360. 215840.]
 Eta north coordinates: [153600. 153600. 153600.]

Xi east coordinates: 230400.0
 Eta east coordinates: [111200. 120320. 120480. 144960. 150400. 150560. 152160.]

Xi south coordinates: [33920. 36160. 36320. 36480. 59360. 61440. 78080. 78240. 78400. 78560.
 78720. 90240. 90400. 90560.]
 Eta south coordinates: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



In [9]:
# mask the corresponding points in mask_rho
coords = (
    (xi_rho_west_coords, eta_rho_west_coords),
    (xi_rho_north_coords, eta_rho_north_coords),
    (xi_rho_east_coords, eta_rho_east_coords),
    (xi_rho_south_coords, eta_rho_south_coords),
)
for coord in coords:
    grid_ds.mask_rho.loc[dict(xi_rho=coord[0], eta_rho=coord[1])] = 0

In [10]:
print_values()

West values: []
North values: [0. 0. 0.]
East values: [0. 0. 0. 0. 0. 0. 0.]
South values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Rewrite other masks

In [13]:
grid_ds.mask_rho

<xarray.DataArray 'mask_rho' (eta_rho: 961, xi_rho: 1441)>
[1384801 values with dtype=float64]
Coordinates:
  * xi_rho   (xi_rho) float64 0.0 160.0 320.0 ... 2.301e+05 2.302e+05 2.304e+05
  * eta_rho  (eta_rho) float64 0.0 160.0 320.0 ... 1.533e+05 1.534e+05 1.536e+05
    lon_rho  (eta_rho, xi_rho) float64 ...
    lat_rho  (eta_rho, xi_rho) float64 ...
Attributes:
    long_name:     mask on RHO-points
    option_0:      land
    option_1:      water
    grid_mapping:  grid_mapping

In [35]:
left_mask = grid_ds.mask_rho.isel(xi_rho=slice(1, None)).values  # left border
right_mask = grid_ds.mask_rho.isel(xi_rho=slice(None, -1)).values  # Right border
u_mask = left_mask * right_mask
grid_ds.mask_u.values = u_mask

In [36]:
bottom_mask = grid_ds.mask_rho.isel(eta_rho=slice(1, None)).values  # bottom border
upper_mask = grid_ds.mask_rho.isel(eta_rho=slice(None, -1)).values  # upper border
v_mask = bottom_mask * upper_mask
grid_ds.mask_v.values = v_mask

In [43]:
psi_mask = u_mask[:-1, :] * v_mask[:, :-1]  # [eta, xi]
grid_ds.mask_psi.values = psi_mask

In [41]:
print(u_mask.shape, v_mask.shape, psi_mask.shape)

(961, 1440) (960, 1441) (960, 1440)


## Save

In [45]:
grid_ds.to_netcdf(path='fram_data/norfjords_160m_grid_fix.nc', format='NETCDF4')

In [51]:
grid_ds.mask_rho.isel(eta_rho=slice(953, 958), xi_rho=slice(1435, 1440))

<xarray.DataArray 'mask_rho' (eta_rho: 5, xi_rho: 5)>
array([[0., 1., 1., 0., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * xi_rho   (xi_rho) float64 2.296e+05 2.298e+05 2.299e+05 2.301e+05 2.302e+05
  * eta_rho  (eta_rho) float64 1.525e+05 1.526e+05 1.528e+05 1.53e+05 1.531e+05
    lon_rho  (eta_rho, xi_rho) float64 ...
    lat_rho  (eta_rho, xi_rho) float64 ...
Attributes:
    long_name:     mask on RHO-points
    option_0:      land
    option_1:      water
    grid_mapping:  grid_mapping

In [53]:
grid_ds.mask_u.isel(eta_u=slice(953, 958), xi_u=slice(1435, 1440))

<xarray.DataArray 'mask_u' (eta_u: 5, xi_u: 5)>
array([[0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 1.],
       [1., 0., 0., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * xi_u     (xi_u) float64 2.297e+05 2.298e+05 2.3e+05 2.302e+05 2.303e+05
  * eta_u    (eta_u) float64 1.525e+05 1.526e+05 1.528e+05 1.53e+05 1.531e+05
    lon_u    (eta_u, xi_u) float64 ...
    lat_u    (eta_u, xi_u) float64 ...
Attributes:
    long_name:     mask on U-points
    option_0:      land
    option_1:      water
    grid_mapping:  grid_mapping

In [54]:
grid_ds.mask_v.isel(eta_v=slice(953, 958), xi_v=slice(1435, 1440))

<xarray.DataArray 'mask_v' (eta_v: 5, xi_v: 5)>
array([[0., 1., 1., 0., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * xi_v     (xi_v) float64 2.296e+05 2.298e+05 2.299e+05 2.301e+05 2.302e+05
  * eta_v    (eta_v) float64 1.526e+05 1.527e+05 1.529e+05 1.53e+05 1.532e+05
    lon_v    (eta_v, xi_v) float64 ...
    lat_v    (eta_v, xi_v) float64 ...
Attributes:
    long_name:     mask on V-points
    option_0:      land
    option_1:      water
    grid_mapping:  grid_mapping

In [55]:
grid_ds.mask_psi.isel(eta_psi=slice(953, 958), xi_psi=slice(1435, 1440))

<xarray.DataArray 'mask_psi' (eta_psi: 5, xi_psi: 5)>
array([[0., 1., 0., 0., 1.],
       [1., 1., 0., 1., 1.],
       [1., 0., 0., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Dimensions without coordinates: eta_psi, xi_psi
Attributes:
    long_name:  mask on PSI-points
    option_0:   land
    option_1:   water